In [1]:
import os
os.chdir('/Users/campshelor/Desktop/GitHub Repos/miros')

In [2]:
import pandas as pd

In [25]:
df["Country"] = df["Country"].str.replace("AT", "Austria", regex=False)
df["Country"] = df["Country"].str.replace("DE", "Germany", regex=False)
df["Country"] = df["Country"].str.replace("ES", "Spain", regex=False)
df["Country"] = df["Country"].str.replace("FI", "Finland", regex=False)
df["Country"] = df["Country"].str.replace("DK", "Denmark", regex=False)
df["Country"] = df["Country"].str.replace("IT", "Italy", regex=False)
df["Country"] = df["Country"].str.replace("PT", "Portugal", regex=False)
df["Country"] = df["Country"].str.replace("SE", "Sweden", regex=False)
df["Country"] = df["Country"].str.replace("FR", "France", regex=False)
df["Country"] = df["Country"].str.replace("US", "USA", regex=False)
df["Country"] = df["Country"].str.replace("USAAA", "USA", regex=False)
df["Country"] = df["Country"].str.replace("USAA", "USA", regex=False)
df["Country"] = df["Country"].str.replace("UK", "United Kingdom", regex=False)


for v in df["Country"].unique():
    print(v)


Austria
Belgium
Canada
Germany
Denmark
Spain
Finland
France
Italy
Netherlands
Norway
Portugal
Sweden
Switzerland
United Kingdom
USA


In [26]:
for v in df["Focus Keywords"].unique():
    print(v)

systematic musicology; music cognition; empirical musicology
music information retrieval; audio feature extraction; machine learning; genre classification; interfaces
systematic musicology; musicology; acoustics/psychoacoustics (unit focus)
machine learning for music, expressive performance, MIR, audio signal processing, recommendation
embodied music cognition, XR, interaction, movement, psychoacoustics, performance
live performance research; music perception & movement; hearing science; health & wellbeing; motion capture; EEG; acoustics
melodic perception; tonality; memory for music; musical structure; amusia; aging & music
rhythm & beat perception; timing; motor system; Parkinson’s disease; fMRI/EEG; brain stimulation; music & movement
music & language; auditory perception; cognitive neuroscience; EEG/MEG/fMRI; amusia; rhythm & pitch
music perception & cognition; audio/music technology; sound recording; instruments & interfaces; interaction & immersive systems; acoustics; MIR
creativ

In [27]:
import pandas as pd
import re

# ----- Your canonical categories -----
canonical = [
    "Music Information Retrieval (MIR)",
    "Music & AI / Machine Learning",
    "Computational Musicology",
    "Audio Signal Processing",
    "Auditory Cognition",
    "Music Perception",
    "Music Emotion & Affective Science",
    "Memory & Learning",
    "Development & Expertise",
    "Music Neuroscience",
    "Cognitive Neuroscience",
    "Music & Health / Therapy",
    "Performance Science",
    "Human–Computer Interaction (Music)",
    "Music Technology & Engineering",
    "Cross-Cultural Music Cognition",
    "Education & Pedagogy",
]

# ----- Mapping from token → canonical category -----
# IMPORTANT: these keys must be LOWERCASE and refer to INDIVIDUAL CONCEPTS
token_map = {
    # MIR
    "mir": "Music Information Retrieval (MIR)",
    "music information retrieval": "Music Information Retrieval (MIR)",
    "information retrieval": "Music Information Retrieval (MIR)",

    # AI/ML
    "machine learning": "Music & AI / Machine Learning",
    "deep learning": "Music & AI / Machine Learning",
    "creative ai": "Music & AI / Machine Learning",
    "ai & music": "Music & AI / Machine Learning",
    "ai/music": "Music & AI / Machine Learning",
    "generative systems": "Music & AI / Machine Learning",
    "machine listening": "Music & AI / Machine Learning",
    "automatic music generation": "Music & AI / Machine Learning",

    # Computational musicology
    "systematic musicology": "Computational Musicology",
    "empirical musicology": "Computational Musicology",
    "computational musicology": "Computational Musicology",
    "digital musicology": "Computational Musicology",
    "corpus analysis": "Computational Musicology",
    "corpus-based analysis": "Computational Musicology",
    "computational music theory": "Computational Musicology",

    # Audio signal processing
    "audio signal processing": "Audio Signal Processing",
    "audio processing": "Audio Signal Processing",
    "audio engineering": "Audio Signal Processing",
    "spatial audio": "Audio Signal Processing",
    "room acoustics": "Audio Signal Processing",
    "acoustics": "Audio Signal Processing",
    "binaural": "Audio Signal Processing",
    "ambisonics": "Audio Signal Processing",

    # Auditory cognition
    "hearing science": "Auditory Cognition",
    "psychoacoustics": "Auditory Cognition",
    "auditory perception": "Auditory Cognition",
    "auditory illusions": "Auditory Cognition",

    # Music perception
    "music perception": "Music Perception",
    "melodic perception": "Music Perception",
    "rhythm perception": "Music Perception",
    "beat perception": "Music Perception",
    "tonality": "Music Perception",
    "musical structure": "Music Perception",

    # Emotion
    "music & emotion": "Music Emotion & Affective Science",
    "affective computing": "Music Emotion & Affective Science",
    "affective responses": "Music Emotion & Affective Science",
    "empirical aesthetics": "Music Emotion & Affective Science",

    # Memory / learning
    "memory for music": "Memory & Learning",
    "music-evoked memory": "Memory & Learning",

    # Development / expertise
    "development": "Development & Expertise",
    "developmental": "Development & Expertise",
    "aging": "Development & Expertise",
    "music training": "Development & Expertise",

    # Music neuroscience
    "music neuroscience": "Music Neuroscience",
    "neuroscience of music": "Music Neuroscience",
    "auditory neuroscience": "Music Neuroscience",

    # Cognitive neuroscience
    "cognitive neuroscience": "Cognitive Neuroscience",
    "neural dynamics": "Cognitive Neuroscience",
    "neural coding": "Cognitive Neuroscience",
    "brain imaging": "Cognitive Neuroscience",
    "neuroimaging": "Cognitive Neuroscience",

    # Health / therapy
    "music therapy": "Music & Health / Therapy",
    "neurologic music therapy": "Music & Health / Therapy",
    "health & wellbeing": "Music & Health / Therapy",
    "clinical applications": "Music & Health / Therapy",

    # Performance science
    "expressive performance": "Performance Science",
    "live performance": "Performance Science",
    "performance science": "Performance Science",

    # HCI / interfaces
    "hci": "Human–Computer Interaction (Music)",
    "interfaces": "Human–Computer Interaction (Music)",
    "musical interfaces": "Human–Computer Interaction (Music)",
    "interactive systems": "Human–Computer Interaction (Music)",
    "xr": "Human–Computer Interaction (Music)",
    "vr": "Human–Computer Interaction (Music)",

    # Music technology
    "audio technology": "Music Technology & Engineering",
    "sound synthesis": "Music Technology & Engineering",
    "electroacoustic music": "Music Technology & Engineering",

    # Education
    "education": "Education & Pedagogy",
    "pedagogy": "Education & Pedagogy",
}

# ----- Helper: split → normalize → map → dedupe → reassemble -----
def clean_keywords(cell):
    if pd.isna(cell):
        return ""

    # 1. split on semicolon, comma, slash
    tokens = re.split(r"[;,/]", str(cell))

    out = []
    for t in tokens:
        t_clean = t.strip().lower()

        # map if recognized
        if t_clean in token_map:
            out.append(token_map[t_clean])

    # dedupe
    out = sorted(set(out))

    # join back
    return "; ".join(out)

df["Focus Keywords Cleaned"] = df["Focus Keywords"].apply(clean_keywords)

df.rename(columns={"Lab Name":"LabName", "Focus Keywords Cleaned":"Keywords", "Focus Keywords":"Specifics", "Leads":"ContactPerson", "Contact":"ContactInfo"})



,LabName,Institution,City,Country,Website,ContactPerson,ContactInfo,Specifics,Sources,Notes,__source_file,Latitude,Longitude,Keywords
0,Centre for Systematic Musicology (ZSM),University of Graz,Graz,Austria,https://zsm.kug.ac.at/,Richard Parncutt (founder/contact),richard.parncutt@uni-graz.at,systematic musicology; music cognition; empiri...,https://zsm.kug.ac.at/,Centre existed 2009–2023 (archival site still ...,music_research_labs_batch_2025-11-12_1333.csv,47.070868,15.438279,Computational Musicology
1,MIR Group @ TU Wien (Information & Software En...,TU Wien,Vienna,Austria,https://www.ifs.tuwien.ac.at/mir/,Andreas Rauber (group lead); Thomas Lidy (coll...,rauber@ifs.tuwien.ac.at,music information retrieval; audio feature ext...,https://informatics.tuwien.ac.at/people/andrea...,Group within Information Systems Engineering (...,music_research_labs_batch_2025-11-12_1333.csv,48.208354,16.372504,Human–Computer Interaction (Music); Music & AI...
2,Systematic Musicology (Department of Musicology),University of Vienna,Vienna,Austria,https://musikwissenschaft.univie.ac.at/,Michele Calella (Head); Anja‑Xiaoxing Cui (Dep...,musikwissenschaft@univie.ac.at,systematic musicology; musicology; acoustics/p...,https://ufind.univie.ac.at/en/pvz_sub.html?id=442,Systematic Musicology is one of the department...,music_research_labs_batch_2025-11-12_1333.csv,48.208354,16.372504,Audio Signal Processing; Computational Musicology
3,Institute of Computational Perception (ICP),Johannes Kepler University Linz,Linz,Austria,NaN,Gerhard Widmer (Head); Markus Schedl,https://www.jku.at/en/institute-of-computation...,"machine learning for music, expressive perform...",https://www.jku.at/en/institute-of-computation...,NaN,music_cognition_labs_part1.csv,48.305908,14.286198,Audio Signal Processing; Music Information Ret...
4,IPEM – Institute for Psychoacoustics & Electro...,Ghent University,Ghent,Belgium,NaN,Pieter-Jan Maes (Director & Head of Research),https://www.ugent.be/lw/kunstwetenschappen/ipe...,"embodied music cognition, XR, interaction, mov...",https://www.ugent.be/lw/kunstwetenschappen/ipe...,NaN,music_cognition_labs_part1.csv,51.053829,3.725012,Auditory Cognition; Human–Computer Interaction...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56,Music Cognition (Eastman School of Music),University of Rochester – Eastman School of Music,"Rochester, NY",USA,https://www.esm.rochester.edu/theory/ea/music-...,David Temperley (contact for Music Cognition),temperley@esm.rochester.edu,music perception; music cognition; music theor...,https://www.esm.rochester.edu/theory/ea/music-...,NaN,Batch__US_Labs__Vanderbilt___Eastman_.csv,NaN,NaN,Music Perception
57,Gentner Lab (Auditory Neuroscience),"University of California, San Diego","San Diego, CA",USA,https://gentnerlab.ucsd.edu/,Timothy Q. Gentner (PI),tgentner@ucsd.edu; Phone: +1 858‑822‑6763; McG...,auditory neuroethology; animal communication; ...,https://gentnerlab.ucsd.edu/; https://psycholo...,NaN,Batch__US_Labs__Vanderbilt___Eastman_.csv,NaN,NaN,Cognitive Neuroscience
58,LAMP Lab – Language & Music Perception,University of San Diego,"San Diego, CA",USA,https://www.lamp-lab.org/,Laura K. Getz (Director),usdlamplab@gmail.com; lgetz@sandiego.edu,music & speech perception; language and memory...,https://www.lamp-lab.org/; https://www.sandieg...,NaN,music_cognition_research_batch_CAN_US.csv,NaN,NaN,Cognitive Neuroscience
59,Media Arts & Technology (MAT) Program (incl. A...,"University of California, Santa Barbara","Santa Barbara, CA",USA,http://www.mat.ucsb.edu/,Program faculty include JoAnn Kuchera‑Morin (A...,MAT program: info@mat.ucsb.edu; Address: 3309 ...,media arts; electronic music; visualization; V...,https://www.mat.ucsb.edu/mad/; https://www.gra...,NaN,Batch__US_Labs__Vanderbilt___Eastman_.csv,NaN,NaN,Audio Signal Processing; Human–Computer Intera...


In [28]:
df.to_csv("data/mir_labs.csv", index=False)

In [29]:
import pandas as pd
import time
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut, GeocoderUnavailable
from geopy.extra.rate_limiter import RateLimiter

# Load your CSV
# Initialize geocoder
geolocator = Nominatim(user_agent="mir-labs-geocoder", timeout=10)

# Rate limiter with retries and delay
geocode = RateLimiter(
    geolocator.geocode,
    min_delay_seconds=1,
    max_retries=3,
    error_wait_seconds=2
)

def safe_geocode(query):
    """Return (lat, lon) or (None, None) on failure."""
    try:
        location = geocode(query)
        if location:
            return location.latitude, location.longitude
        else:
            return None, None
    except (GeocoderTimedOut, GeocoderUnavailable):
        return None, None
    except Exception:
        return None, None

# Only geocode missing rows
if "Latitude" not in df.columns:
    df["Latitude"] = None
if "Longitude" not in df.columns:
    df["Longitude"] = None

for idx, row in df.iterrows():
    if pd.isna(row["Latitude"]) or pd.isna(row["Longitude"]):
        query = f"{row['City']}, {row['Country']}"
        print(f"Geocoding: {query}")
        lat, lon = safe_geocode(query)
        df.at[idx, "Latitude"] = lat
        df.at[idx, "Longitude"] = lon
        time.sleep(1)  # extra politeness

# Save updated file
df.to_csv("mir_labs_with_coords.csv", index=False)

df.head()


Geocoding: Espoo (Helsinki metro), Finland
Geocoding: London, United Kingdom
Geocoding: Plymouth, United Kingdom
Geocoding: Sheffield, United Kingdom
Geocoding: Southampton, United Kingdom
Geocoding: York, United Kingdom
Geocoding: Cambridge, United Kingdom
Geocoding: Durham, United Kingdom
Geocoding: London, United Kingdom
Geocoding: London, United Kingdom
Geocoding: Atlanta, GA, USA
Geocoding: Berkeley, CA, USA
Geocoding: Cambridge, MA, USA
Geocoding: Davis, CA, USA
Geocoding: New York, NY, USA
Geocoding: Princeton, NJ, USA
Geocoding: Stanford, CA, USA
Geocoding: Columbus, OH, USA
Geocoding: Evanston, IL, USA
Geocoding: Los Angeles, CA, USA
Geocoding: Medford, MA, USA
Geocoding: Nashville, TN, USA
Geocoding: New York, NY, USA
Geocoding: Rochester, NY, USA
Geocoding: Rochester, NY, USA
Geocoding: San Diego, CA, USA
Geocoding: San Diego, CA, USA
Geocoding: Santa Barbara, CA, USA
Geocoding: Storrs, CT, USA


,Lab Name,Institution,City,Country,Website,Leads,Contact,Focus Keywords,Sources,Notes,__source_file,Latitude,Longitude,Focus Keywords Cleaned
0,Centre for Systematic Musicology (ZSM),University of Graz,Graz,Austria,https://zsm.kug.ac.at/,Richard Parncutt (founder/contact),richard.parncutt@uni-graz.at,systematic musicology; music cognition; empiri...,https://zsm.kug.ac.at/,Centre existed 2009–2023 (archival site still ...,music_research_labs_batch_2025-11-12_1333.csv,47.070868,15.438279,Computational Musicology
1,MIR Group @ TU Wien (Information & Software En...,TU Wien,Vienna,Austria,https://www.ifs.tuwien.ac.at/mir/,Andreas Rauber (group lead); Thomas Lidy (coll...,rauber@ifs.tuwien.ac.at,music information retrieval; audio feature ext...,https://informatics.tuwien.ac.at/people/andrea...,Group within Information Systems Engineering (...,music_research_labs_batch_2025-11-12_1333.csv,48.208354,16.372504,Human–Computer Interaction (Music); Music & AI...
2,Systematic Musicology (Department of Musicology),University of Vienna,Vienna,Austria,https://musikwissenschaft.univie.ac.at/,Michele Calella (Head); Anja‑Xiaoxing Cui (Dep...,musikwissenschaft@univie.ac.at,systematic musicology; musicology; acoustics/p...,https://ufind.univie.ac.at/en/pvz_sub.html?id=442,Systematic Musicology is one of the department...,music_research_labs_batch_2025-11-12_1333.csv,48.208354,16.372504,Audio Signal Processing; Computational Musicology
3,Institute of Computational Perception (ICP),Johannes Kepler University Linz,Linz,Austria,NaN,Gerhard Widmer (Head); Markus Schedl,https://www.jku.at/en/institute-of-computation...,"machine learning for music, expressive perform...",https://www.jku.at/en/institute-of-computation...,NaN,music_cognition_labs_part1.csv,48.305908,14.286198,Audio Signal Processing; Music Information Ret...
4,IPEM – Institute for Psychoacoustics & Electro...,Ghent University,Ghent,Belgium,NaN,Pieter-Jan Maes (Director & Head of Research),https://www.ugent.be/lw/kunstwetenschappen/ipe...,"embodied music cognition, XR, interaction, mov...",https://www.ugent.be/lw/kunstwetenschappen/ipe...,NaN,music_cognition_labs_part1.csv,51.053829,3.725012,Auditory Cognition; Human–Computer Interaction...


In [7]:
df_new = pd.read_csv('data/mir_labs_with_coords.csv')

df_new.rename(columns={"Focus Keywords":"Specifics","Focus Keywords Cleaned":"Keywords", "Leads":"ContactPerson","Contact":"ContactInfo"}, inplace=True)

df_new.head()
df_new.to_csv("data/mir_labs.csv", index=False)

In [8]:
df_test = pd.read_csv("data/mir_labs.csv")
df_test.head()

,Lab Name,Institution,City,Country,Website,ContactPerson,ContactInfo,Specifics,Sources,Notes,__source_file,Latitude,Longitude,Keywords
0,Centre for Systematic Musicology (ZSM),University of Graz,Graz,Austria,https://zsm.kug.ac.at/,Richard Parncutt (founder/contact),richard.parncutt@uni-graz.at,systematic musicology; music cognition; empiri...,https://zsm.kug.ac.at/,Centre existed 2009–2023 (archival site still ...,music_research_labs_batch_2025-11-12_1333.csv,47.070868,15.438279,Computational Musicology
1,MIR Group @ TU Wien (Information & Software En...,TU Wien,Vienna,Austria,https://www.ifs.tuwien.ac.at/mir/,Andreas Rauber (group lead); Thomas Lidy (coll...,rauber@ifs.tuwien.ac.at,music information retrieval; audio feature ext...,https://informatics.tuwien.ac.at/people/andrea...,Group within Information Systems Engineering (...,music_research_labs_batch_2025-11-12_1333.csv,48.208354,16.372504,Human–Computer Interaction (Music); Music & AI...
2,Systematic Musicology (Department of Musicology),University of Vienna,Vienna,Austria,https://musikwissenschaft.univie.ac.at/,Michele Calella (Head); Anja‑Xiaoxing Cui (Dep...,musikwissenschaft@univie.ac.at,systematic musicology; musicology; acoustics/p...,https://ufind.univie.ac.at/en/pvz_sub.html?id=442,Systematic Musicology is one of the department...,music_research_labs_batch_2025-11-12_1333.csv,48.208354,16.372504,Audio Signal Processing; Computational Musicology
3,Institute of Computational Perception (ICP),Johannes Kepler University Linz,Linz,Austria,NaN,Gerhard Widmer (Head); Markus Schedl,https://www.jku.at/en/institute-of-computation...,"machine learning for music, expressive perform...",https://www.jku.at/en/institute-of-computation...,NaN,music_cognition_labs_part1.csv,48.305908,14.286198,Audio Signal Processing; Music Information Ret...
4,IPEM – Institute for Psychoacoustics & Electro...,Ghent University,Ghent,Belgium,NaN,Pieter-Jan Maes (Director & Head of Research),https://www.ugent.be/lw/kunstwetenschappen/ipe...,"embodied music cognition, XR, interaction, mov...",https://www.ugent.be/lw/kunstwetenschappen/ipe...,NaN,music_cognition_labs_part1.csv,51.053829,3.725012,Auditory Cognition; Human–Computer Interaction...


In [9]:
df_new["Keywords"].unique()

array(['Computational Musicology',
       'Human–Computer Interaction (Music); Music & AI / Machine Learning; Music Information Retrieval (MIR)',
       'Audio Signal Processing; Computational Musicology',
       'Audio Signal Processing; Music Information Retrieval (MIR); Performance Science',
       'Auditory Cognition; Human–Computer Interaction (Music)',
       'Audio Signal Processing; Auditory Cognition; Music & Health / Therapy',
       'Memory & Learning; Music Perception', nan,
       'Auditory Cognition; Cognitive Neuroscience',
       'Audio Signal Processing; Music Information Retrieval (MIR)',
       'Music & AI / Machine Learning; Music Emotion & Affective Science',
       'Music & Health / Therapy', 'Auditory Cognition; Music Perception',
       'Auditory Cognition; Music & Health / Therapy; Music Emotion & Affective Science',
       'Audio Signal Processing; Music Technology & Engineering',
       'Audio Signal Processing; Auditory Cognition; Computational Musicology',
